In [1]:
import pandas as pd
import numpy as np
import re

pd.set_option('display.max_columns', None)

In [40]:
df_lyrics2 = pd.read_csv('files/lyrics.csv')
df_last_album = pd.read_csv('files/lyrics_new_album.csv', index_col=0)


In [41]:
df_genres = pd.read_csv('files/genres_to_songs.csv')

In [42]:
df_last_album['lyrics'] = df_last_album['lyrics'].apply(lambda x : re.sub(r'\[.*?\]', '', x).strip().replace('\n', '  '))


In [43]:
df_last_album['Album'] = 'The Tortured Poets Department'

In [44]:
df_last_album = df_last_album.reindex(columns = ['song', 'Album', 'lyrics'])

In [45]:
df_last_album.columns = ['Song', 'Album', 'Lyrics']

In [46]:
df_lyrics2['Album'].unique()

array(['Red (Taylor’s Version)', 'Lover', '1989 (Deluxe)',
       'Midnights (3am Edition)', 'Taylor Swift', 'Speak Now',
       'Cats: Highlights From the Motion Picture Soundtrack',
       'Fearless (Taylor’s Version)', 'reputation', 'evermore',
       'The Taylor Swift Holiday Collection - EP',
       'Women in Music Pt. III (Expanded Edition)',
       'Two Lanes of Freedom (Accelerated Deluxe)',
       'Midnights (Target Exclusive)',
       'Fifty Shades Darker (Original Motion Picture Soundtrack)',
       'Speak Now (Deluxe)', 'Taylor Swift (Best Buy Exclusive)', nan,
       'How Long Do You Think It’s Gonna Last?',
       'The Hunger Games: Songs from District 12 and Beyond',
       'One Chance (Original Motion Picture Soundtrack)',
       '1989 (Taylor’s Version)', 'Love Drunk',
       'Hannah Montana: The Movie',
       'Carolina (From The Motion Picture “Where The Crawdads Sing”) - Single',
       'Christmas Tree Farm', 'folklore', 'folklore (deluxe version)',
       'evermore

In [47]:
albumes = ['Red (Taylor’s Version)', 
           'Lover',
           '1989 (Deluxe)',
           'Midnights (3am Edition)',
           'Taylor Swift',
           'Speak Now (Deluxe)',
           'Fearless (Taylor’s Version)',
           'reputation',
           'evermore (deluxe version)',
           'folklore (deluxe version)',
           'folklore',
           'evermore',
           'Speak Now',
           '1989 (Taylor’s Version)']

df_lyrics_filtrado = df_lyrics2[df_lyrics2['Album'].isin(albumes)]

In [48]:
diccionario = {}

for track in df_lyrics_filtrado['Song']:
    
    df_especific = df_lyrics_filtrado[df_lyrics_filtrado['Song'] == track]
    cadena = "  ".join(df_especific['Lyric'])
    diccionario[track] = cadena

In [49]:
df_lyrics_clean = df_lyrics_filtrado.drop_duplicates(subset = 'Song')

In [50]:
lista_a_dropear = []

for cancion in df_lyrics_clean['Song']:
    
    if '[From the Vault]' in cancion:
        lista_a_dropear.append(cancion)

In [51]:
df_lyrics_clean = df_lyrics_clean[~df_lyrics_clean['Song'].isin(lista_a_dropear)]

In [52]:
df_lyrics_clean['Lyrics'] = df_lyrics_clean.apply(lambda x : diccionario[x['Song']], axis=1)

In [53]:
df_lyrics_clean.drop(columns = ['Lyric', 'Previous Lyric', 'Next Lyric', 'Multiplicity'], inplace=True)

In [54]:
df_lyrics_clean = df_lyrics_clean.reset_index().drop('index', axis=1)

In [55]:
df_lyrics = pd.concat([df_lyrics_clean, df_last_album], ignore_index=True)

In [56]:
df_lyrics['Song'] = df_lyrics['Song'].apply(lambda x : x.replace(' (Taylor’s Version)', ''))
df_lyrics['Song'] = df_lyrics['Song'].apply(lambda x : x.replace('(Taylor’s Version)', ''))
df_lyrics['Album'] = df_lyrics['Album'].apply(lambda x : x.replace(' (Taylor’s Version)', ''))
df_lyrics['Album'] = df_lyrics['Album'].apply(lambda x : x.replace(' (Deluxe)', ''))
df_lyrics['Song'] = df_lyrics['Song'].apply(lambda x : x.replace(' (Pop Version)', ''))
df_lyrics['Song'] = df_lyrics['Song'].apply(lambda x : x.replace('(Radio Edit)', ''))

In [57]:
diccionario_cambios = {'...Ready for It?' : '…Ready for It?',
                       'So It Goes...' : 'So It Goes…',
                       '’tis the damn season' : '‘tis the damn season',
                       'l​ong story short' : 'long story short',
                       "Who's Afraid of Little Old Me?" : 'Who’s Afraid of Little Old Me?',
                       'You’re Not Sorry' : "You're Not Sorry",
                       'You’re On Your Own, Kid' : "You're on Your Own, Kid",
                       'Would’ve, Could’ve, Should’ve' : "Would've, Could've, Should've"}

df_lyrics['Song'] = df_lyrics['Song'].map(diccionario_cambios).fillna(df_lyrics['Song'])

In [58]:
words = ['In ', 'On ', 'At ', 'By ', 'For ', 'With ', 'From ', 'To ', 'Into ', 'Onto ', 'Of ', 'About ', 'Above ', 'Below ', 'Under ', 'Over ', 'Through ', 'Between ', 'Among ', 'Within ', 'The ', 'But ', 'A ', 'Oh ', 'Oh,', 'It ', 'If ', 'How ', 'Where ', 'When ', 'What ', 'Who ', 'Why ', 'But ', 'Because ', 'An ', 'And ', 'May ']
pattern = r"'[^ ]*"
df_lyrics['Lyrics'] = df_lyrics['Lyrics'].apply(lambda x : re.sub(pattern, '', x))

for word in words:
    df_lyrics['Lyrics'] = df_lyrics['Lyrics'].str.replace(word , ' ')
    df_lyrics['Lyrics'] = df_lyrics['Lyrics'].str.replace(f' {word.lower()}' , ' ')


In [59]:
df_lyrics['song'] = df_lyrics['Song'].str.lower().str.strip()
df_genres['song'] = df_genres['song'].str.lower().str.strip()

In [60]:
df_genres = df_genres.merge(df_lyrics, left_on='song', right_on='song', how='left')

In [61]:
df_genres[df_genres['Song'].isnull()]['song'].unique()

array([], dtype=object)

In [62]:
df_genres.drop(['song', 'album'], axis=1, inplace=True)

In [63]:
df_genres = df_genres.reindex(columns = ['Album', 'Song', 'genre', 'Lyrics'])

In [64]:
df_genres = df_genres.sort_values('Song', ascending=True)

In [65]:
df_genres

,Album,Song,genre,Lyrics
68,Red,22,Fiestas y Tradiciones,feels like perfect night dress up like hips...
57,Red,22,Música disco,feels like perfect night dress up like hips...
74,Red,22,Dance pop,feels like perfect night dress up like hips...
75,Red,22,Bubblegum pop,feels like perfect night dress up like hips...
51,Red,22,Pop,feels like perfect night dress up like hips...
...,...,...,...,...
101,reputation,…Ready for It?,Pop,Knew he was killer first time that I saw him ...
107,reputation,…Ready for It?,Música infantil,Knew he was killer first time that I saw him ...
110,reputation,…Ready for It?,Electropop,Knew he was killer first time that I saw him ...
125,reputation,…Ready for It?,Música industrial,Knew he was killer first time that I saw him ...


In [77]:
#df_genres.to_csv('files/taylor_full_lyrics_w_genre.csv', index=False)

## Limpieza DF Twitter

In [67]:
df_tuits = pd.read_csv('files/TaylorSwift13.csv')

In [68]:
df_tuits.drop('created_at', axis=1, inplace=True)

In [69]:
df_tuits['date'] = df_tuits['date'].apply(lambda x : x.replace('+00:00', ''))

In [70]:
df_tuits['media'] = df_tuits['media'].apply(lambda x : 1 if isinstance(x, str) else 0)
df_tuits['outlinks'] = df_tuits['outlinks'].apply(lambda x : 0 if x == '[]' else 1)
df_tuits['quotedTweet'] = df_tuits['quotedTweet'].apply(lambda x : 1 if isinstance(x, str) else 0)
df_tuits['retweetedTweet'] = df_tuits['retweetedTweet'].apply(lambda x : 1 if isinstance(x, str) else 0)

df_tuits['content'] = df_tuits['content'].apply(lambda x : x.replace('\n', ' '))

In [71]:
df_tuits['users'] = df_tuits.apply(lambda x : re.findall(r'@\S+', x['content']), axis=1)

In [72]:
df_citados = df_tuits[['content', 'date', 'users']]

In [73]:
df_tuits.drop('users', axis=1, inplace=True)

In [74]:
df_citados = df_citados.explode('users')

In [75]:
df_citados.dropna(subset='users', inplace = True)

In [76]:
#df_tuits.to_csv('files/taylor_tuits_limpio.csv', index=False)
#df_citados.to_csv('files/taylor_tuits_citados.csv', index=False)

## Creación tabla extra spoti

In [20]:
df = pd.read_csv('files/all_songs_audio_data.csv', index_col=0)

In [21]:
df_auxiliar = df.groupby('album')[['acousticness', 'danceability', 'energy', 'instrumentalness',
       'liveness', 'loudness', 'speechiness', 'tempo', 'valence']].mean().reset_index()

In [22]:
diccionario = {'album':[], 'category': [], 'mean':[]}

In [23]:
df_auxiliar

,album,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence
0,1989 (Deluxe Edition),0.244620,0.633211,0.624842,0.000704,0.203211,-7.924105,0.173453,127.033053,0.454211
1,Fearless Platinum Edition,0.202448,0.575947,0.601053,0.000002,0.159826,-5.708211,0.031905,123.089263,0.380053
2,Lover,0.333743,0.658222,0.545222,0.000733,0.115233,-8.013278,0.099117,119.972722,0.481444
3,Midnights (3am Edition),0.413220,0.626650,0.451150,0.018134,0.150235,-10.588050,0.092075,119.744950,0.282530
4,Red (Deluxe Edition),0.148799,0.633364,0.600773,0.001802,0.126214,-7.380045,0.036577,110.296727,0.468191
5,Speak Now (Deluxe Edition),0.249275,0.556450,0.647900,0.000046,0.172500,-4.726250,0.034670,136.375600,0.416950
6,THE TORTURED POETS DEPARTMENT: THE ANTHOLOGY,0.580152,0.527903,0.460613,0.000214,0.150045,-9.381258,0.056729,122.102097,0.318355
7,Taylor Swift,0.182995,0.545267,0.664267,0.000054,0.160773,-4.731667,0.032687,126.053800,0.426533
8,evermore (deluxe version),0.794118,0.526824,0.494059,0.020617,0.113588,-9.781588,0.057906,120.707294,0.433529
9,folklore (deluxe version),0.717588,0.541941,0.415824,0.000302,0.110465,-10.336059,0.039482,119.884412,0.361353


In [24]:
melted_df = pd.melt(df_auxiliar, id_vars=["album"], value_vars=['acousticness', 'danceability', 'energy', 'instrumentalness',
       'liveness', 'loudness', 'speechiness', 'tempo', 'valence'], var_name="categoria", value_name="promedio")

In [25]:
melted_df

,album,categoria,promedio
0,1989 (Deluxe Edition),acousticness,0.244620
1,Fearless Platinum Edition,acousticness,0.202448
2,Lover,acousticness,0.333743
3,Midnights (3am Edition),acousticness,0.413220
4,Red (Deluxe Edition),acousticness,0.148799
...,...,...,...
94,THE TORTURED POETS DEPARTMENT: THE ANTHOLOGY,valence,0.318355
95,Taylor Swift,valence,0.426533
96,evermore (deluxe version),valence,0.433529
97,folklore (deluxe version),valence,0.361353


In [11]:
melted_df['album'].unique()

array(['1989 (Deluxe Edition)', 'Fearless Platinum Edition', 'Lover',
       'Midnights (3am Edition)', 'Red (Deluxe Edition)',
       'Speak Now (Deluxe Edition)',
       'THE TORTURED POETS DEPARTMENT: THE ANTHOLOGY', 'Taylor Swift',
       'evermore (deluxe version)', 'folklore (deluxe version)',
       'reputation'], dtype=object)

In [26]:
nombres_albumes = {'1989 (Deluxe Edition)' : '1989',
                   'Fearless Platinum Edition' : 'Fearless',
                   'Midnights (3am Edition)' : 'Midnights',
                   'Lover' : 'Lover',
                   'Red (Deluxe Edition)' : 'Red',
                   'Speak Now (Deluxe Edition)' : 'Speak Now',
                   'THE TORTURED POETS DEPARTMENT: THE ANTHOLOGY' : 'The Tortured Poets Department',
                   'Taylor Swift' : 'Taylor Swift',
                   'evermore (deluxe version)' : 'Everore',
                   'folklore (deluxe version)' : 'Folklore',
                   'reputation' : 'Reputation'}

nombres_categorias = {'acousticness' : 'acústico',
                      'danceability' : 'danzabilidad',
                      'energy' : 'energía',
                      'instrumentalness' : 'instrumental',
                      'liveness' : 'directo',
                      'loudness' : 'volumen',
                      'speechiness' : 'liricismo',
                      'tempo' : 'tempo',
                      'valence' : 'positividad'                      
                      }

#melted_df['album'] = melted_df['album'].map(nombres_albumes)
#melted_df['categoria'] = melted_df['categoria'].map(nombres_categorias)

melted_df['album'] = melted_df['album'].apply(lambda x: nombres_albumes.get(x, x))
melted_df['categoria'] = melted_df['categoria'].apply(lambda x: nombres_categorias.get(x, x))



In [27]:
melted_df

,album,categoria,promedio
0,1989,acústico,0.244620
1,Fearless,acústico,0.202448
2,Lover,acústico,0.333743
3,Midnights,acústico,0.413220
4,Red,acústico,0.148799
...,...,...,...
94,The Tortured Poets Department,positividad,0.318355
95,Taylor Swift,positividad,0.426533
96,Everore,positividad,0.433529
97,Folklore,positividad,0.361353


In [28]:
melted_df.to_csv('files/all_songs_audio_data_radar.csv', index = False)